# <center> RESPIRATION RATE ESTIMATION
 
 **08/02/2023**   
    
 **Group 7:** **Giovanni Zago**, **Enrico Lupi**, **Emanuele Sarte**, **Alessio Saccomani** 
 
   The aim of this project is to estimate the Respiratory Rate (RR) by Seismocardiography(SCG), a technique where the detector is positioned above sternum, and Ballistocardiography (BCG), in which there is no contact between the sensor and the body: in this study the sensor was placed....). The measurements were taken using the detector MuSe (Multi-sensor miniaturized, low-power, wireless Inertial Measurement Unit), provided by 221e (https://www.221e.com). An IMU is a combination of an accelerometer and a gyroscope sensor, capable of detecting movements and measuring their intensity in terms of acceleration and rotational speeds. Sometimes, like in this case, a magnetometer is also included. 

In [1]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://www.221e.com/wp-content/uploads/2022/10/221e-Muse1.jpg", width=400, height=300)


In [2]:
Image(url= "https://embeddedinventor.com/wp-content/uploads/2019/07/imu1.jpg?ezimgfmt=ng:webp/ngcb7")

# Download data and first steps

First, we imported the necessary libraries for the analysis (pandas, numpy, matplotlib, scipy, ecc.). 

We then downloaded the datafile 'center_sternum.txt' as a pandas dataframe. For the following analysis we decided to take into account the linear acceleration (in mg), the angular velocity (in degrees per second) and the magnetic field (in mgauss) in all three directions; the quaternions were not considered, instead, so they were dropped. Afterwards, we calibrated the measurements using the information in the file README_1.txt and we added to the dataset the absolute time at which every measuremnt is taken: as the data collection frequency is 200 Hz, each measurement is taken every 5 ms. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy import fft
from scipy import signal
from scipy import optimize
from scipy import linalg
import seaborn as sns
import pywt

In [ ]:
file_name = "center_sternum.txt"
raw_data = pd.read_csv(file_name, sep="\t")

# We drop the columns we're not going to use
raw_data = raw_data.drop(["Log Mode", "qw", "qi", "qj", "qk"], axis=1)

# We look that the frequence is the same in all the dataset
if len(raw_data["Log Freq"].unique()) != 1:
    print("There is more than a frequency")
    exit(1)

# We select the frequency (200 Hz)
ACQ_FREQ = raw_data.loc[0, "Log Freq"] #type: int

# We create a column with the absolute time by multiplying 1/200 s by integers
raw_data.insert(2, "Abs Time", np.arange(0, len(raw_data)) * (1 / ACQ_FREQ), allow_duplicates=False)

raw_data


In [ ]:
offset_gyro = np.array([-2.242224, 2.963463, -0.718397])
calibration_acc = np.array([[1.000966, -0.002326418, -0.0006995499],
                            [-0.002326379, 0.9787045, -0.001540918],
                            [-0.0006995811, -0.001540928, 1.00403]])
offset_acc = np.array([-3.929942, -13.74679, 60.67546])
calibration_magn = np.array([[0.9192851, -0.02325168, 0.003480837],
                             [-0.02325175, 0.914876, 0.004257396],
                             [0.003481006, 0.004257583, 0.8748001]])
offset_magn = np.array([-95.67974, -244.9142, 17.71132])

raw_data[['AccX', 'AccY', 'AccZ']] = np.dot(raw_data[['AccX', 'AccY', 'AccZ']], calibration_acc.T) + offset_acc.T
raw_data[['GyroX', 'GyroY', 'GyroZ']] = raw_data[['GyroX', 'GyroY', 'GyroZ']] + offset_gyro.T
raw_data[['MagnX', 'MagnY', 'MagnZ']] = np.dot(raw_data[['MagnX', 'MagnY', 'MagnZ']], calibration_magn.T) + offset_magn.T
raw_data


# Data visualization

Now, we plot the accelerations, the angular velocity and the magnetic field with respect to the time. In this way we have a first idea about the data, in particular we understand which windows of time is ideal to studying the dataset. We look for times where there's not anomalous variation of acceleration or velocity (maybe some movements of the body independent from the hearth rate, and respiration rate) that we do not want to consider. In the left column there is the entire measure and in the right column there is a zoom to see a smaller window of time.
We see that from 10 seconds to 65 seconds all the measures have a constant behaviour, so we consider this to be the time window we're going to use in the rest of the project. In the plot the window of times is highlighted.

In [ ]:

# # labels = ["GyroX", "GyroY", "GyroZ", "AccX", "AccY", "AccZ"]
# Nvar = len(labels)

# fig, axs = plt.subplots(Nvar, 2, figsize=(18, 4 * Nvar))
# for i in range(2):
#     for j in range(Nvar):

#         if i == 0:
#             x_range = raw_data["Abs Time"]
#             data_range = raw_data[labels[j]]
#         else:
#             ixmin, ixmax = (20 * ACQ_FREQ - 50,  24 * ACQ_FREQ + 50)
#             data_range = raw_data.loc[ixmin:ixmax ,labels[j]]
#             x_range = raw_data.loc[ixmin:ixmax, "Abs Time"]
#             axs[j][i].set_xlim(20, 24)

#         axs[j][i].plot(x_range, data_range, label=labels[j])
#         axs[j][i].set_xlabel("Time [s]")
#         axs[j][i].set_ylabel("Magnitude")
#         axs[j][i].legend(loc="best")



In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter

labels = ["GyroX", "GyroY", "GyroZ", "AccX", "AccY", "AccZ", "MagnX", "MagnY", "MagnZ"]
Nvar = len(labels)

T1_CUT = 10     
IT1_CUT = round(T1_CUT * ACQ_FREQ)
T2_CUT = 65 # included
IT2_CUT = round(T2_CUT * ACQ_FREQ) + 1

plt.figure(figsize=(18, 4 * (Nvar // 3)))
for i in range(Nvar // 3):
    ax = plt.subplot(Nvar // 3, 1, i + 1)
    plt.plot(raw_data["Abs Time"], raw_data[labels[i * 3]], label=labels[i * 3])
    plt.plot(raw_data["Abs Time"], raw_data[labels[i * 3 + 1]], label=labels[i * 3 + 1])
    plt.plot(raw_data["Abs Time"], raw_data[labels[i * 3 + 2]], label=labels[i * 3 + 2])
    ymin, ymax = plt.ylim()
    plt.vlines(x=raw_data.loc[IT1_CUT, "Abs Time"], ymin=ymin, ymax=ymax, colors="black")
    plt.vlines(x=raw_data.loc[IT2_CUT, "Abs Time"], ymin=ymin, ymax=ymax, colors="black")

    ax.fill_between(raw_data.loc[IT1_CUT: IT2_CUT, "Abs Time"], ymin, ymax, color='C0', alpha=0.25)

    ax.xaxis.set_major_locator(MultipleLocator(10))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    plt.xlabel("Time (s)")
    plt.ylabel("Magnitude")
    plt.legend()

plt.show()
raw_data_cut = raw_data.loc[IT1_CUT: IT2_CUT - 1, :]
raw_data_cut

In [ ]:

fig, axs = plt.subplots(Nvar, 2, figsize=(18, 4 * Nvar))
for i in range(2):
    for j in range(Nvar):
        if i == 0:
            x_range = raw_data["Abs Time"]
            data_range = raw_data[labels[j]]
        else:
            xmax, xmin = 16, 24
            ixmin, ixmax = (xmax * ACQ_FREQ - 50,  xmin * ACQ_FREQ + 50)
            data_range = raw_data.loc[ixmin:ixmax ,labels[j]]
            x_range = raw_data.loc[ixmin:ixmax, "Abs Time"]
            axs[j][i].set_xlim(xmax, xmin)

        axs[j][i].plot(x_range, data_range, label=labels[j]) #c='b' ?
        axs[j][i].set_xlabel("Time [s]")

        if "Acc" in labels[j]:
            axs[j][i].set_ylabel("Linear Acceleration [mg]")
        elif "Gyro" in labels[j]:
            axs[j][i].set_ylabel("Angular Velocity [dps]")
        elif "Magn" in labels[j]:
            axs[j][i].set_ylabel("Magnetic Field [mgauss]")

        # axs[j][i].legend(loc="best")


# Statistical Analysis

Now we print for all the columns remained the mean, standard deviation, minimum, maximum and the 25,50,75 percentiles in the time window we have selected. Then we also normalize the data by subtracting the mean and by dividing for the standard deviation. For the accelerations and the angular velocities the 25 and 75 percentiles are nearly simmetric, so the variables change quite regularly. We notice a higher deviation for the acceleration over X, and the angular velocity along Z. We plot also the normalized data in a 9x9 grid in order to notice some correlation but in our case nothing interesting appears

In [ ]:
raw_data_cut.drop(["Log Freq", "Timestamp", "Abs Time"], axis=1).describe()

In [ ]:
data_std = (raw_data_cut[labels] - np.mean(raw_data_cut[labels], axis=0)) / np.std(raw_data_cut[labels], axis=0)
data_std.insert(0, "Abs Time", raw_data_cut["Abs Time"])
data_std.set_index(np.arange(0, len(data_std)), inplace=True)
display(data_std)

# PCA

Now we're compute a PCA (principal component analysis) on the entire dataset (9 columns of data). We select the six components that allow us to maintain the 85% of the variability of the dataset

In [ ]:
avls, avts = linalg.eig(np.cov(data_std[labels].T))
sort_perm = np.flip(np.argsort(avls))

avls = np.real_if_close(avls[sort_perm])
avts = avts[:, sort_perm]

var_ratios = avls / np.sum(avls)
print('Eigenvalues:\n', np.round(avls, 4)) #todo tenere la troncatura?
print('Variability ratios:\n', np.round(var_ratios * 100, 2))
print("Eigenvector:")
display(pd.DataFrame(avts, columns=[f"Avt {i}" for i in range(len(avls))]))

In [ ]:
PCA_LABELS = [f"PC{i+1}" for i in range(len(avls))]
data_rot = pd.DataFrame(data=np.dot(avts.T, data_std[labels].T).T, columns=PCA_LABELS)

perc_soil = .85
N_PCA = np.argmax(np.cumsum(var_ratios) >= perc_soil) + 1

print(f'To keep {int(perc_soil * 100)}% of our data we need {N_PCA} of the {len(var_ratios)} principal components')

display(data_rot)

PCA_LABELS = PCA_LABELS[:N_PCA]
data_pca = data_rot[data_rot.columns[:N_PCA]]
data_pca.insert(0, "Abs Time", data_std["Abs Time"])
display(data_pca)


# Filter

Now we're going to perform a fourier analysis using the scipy fftpack library; in this way we would like to estrapolate the principal frequencies of the data and see if we can see a frequence near the respiratory rate (and also the hearth rate frequency). Firstly we calculate the power and the frequencies and we plot the power w.r.t. the frequency for every variable. In the left column we can see the entire spectrum (with both the respiration's and the hearth's frequencies) while in the right column there is a zoom in the [0,1] frequency

In [ ]:
sig_fft = fft.fftshift(fft.fft(data_pca[PCA_LABELS], axis=0))
sample_freq = fft.fftshift(fft.fftfreq(len(data_pca), d=1/ACQ_FREQ))

plt.figure(figsize=(18, 5 * N_PCA))
for i in range(N_PCA):
    for j in range(2):
        ax = plt.subplot(N_PCA, 2, i * 2 + j + 1)
        plt.plot(sample_freq, np.abs(sig_fft[:, i]), label=PCA_LABELS[i])
        if j == 0:
            plt.xlim([0, 3])
        else:
            plt.xlim([0, 1])
            plt.xticks(np.arange(0, 1, 0.1))
        
        plt.xlabel("Freq [Hz]")
        plt.ylabel("Power")
        plt.legend(loc="best")
    

In [ ]:
total_FFT = np.sum(np.abs(sig_fft), axis=1)
plt.plot(sample_freq, total_FFT)
plt.xlabel("Frequency [Hz]")
plt.ylabel("Power")
plt.xlim([0,1]);

Let's now perform a wavelet transform analysis

In [ ]:
pywt.dwt_max_level(len(data_pca), "sym5")

In [ ]:
#Filter signal using wavelets
#We used db2, with a high decomposition level 
#as we are interested only in low frequencies but we want to exclude those near zero
#we will thus use the last three detail coefficients to
#cover ranges [0.098, 0.78125] Hz 

lvl = 0
if file_name == "center_sternum.txt":
    lvl = 10
else:
    lvl = 9

coeffs = pywt.wavedec(data_pca[PCA_LABELS], "sym5", level=lvl, axis=0)
coeffs[0] = np.zeros_like(coeffs[0])
for i in range(3, lvl + 1):
    coeffs[i] = np.zeros_like(coeffs[i])

A10 = pywt.waverec(coeffs, "sym5", axis=0)


In [ ]:
#plot only approximation wavelet
fig, axs = plt.subplots(nrows=N_PCA, ncols=2, figsize=(18, N_PCA*5))

for i in range(N_PCA):
    for j in range(2):
        #axs[i][j].plot(data_std["Abs Time"], data_std[labels[i]], label=("Original "+labels[i]))
        #axs[i][j].plot(data_std["Abs Time"], filtered[i], label=("Butterworth "+labels[i]))
        #axs[i][j].plot(data_std["Abs Time"], A7[i][:-1],  label=("Butterworth+Wavelet "+labels[i]))
        axs[i][j].plot(data_pca["Abs Time"], A10[:-1, i], label=("Wavelet "+PCA_LABELS[i]))
        if j == 1:
            axs[i][j].set_xlim([20,24])
        
        axs[i][j].set_xlabel("Time [s]")
        axs[i][j].set_ylabel("Magnitude") # ???
        axs[i][j].legend(loc="best")
    

In [ ]:
sig_fft_wt = fft.fftshift(fft.fft(A10[:-1, :], axis=0))
sample_freq = fft.fftshift(fft.fftfreq(len(data_pca), d=1/ACQ_FREQ))

plt.figure(figsize=(18, 5 * N_PCA))
for i in range(N_PCA):
    for j in range(2):
        ax = plt.subplot(N_PCA, 2, i * 2 + j + 1)
        plt.plot(sample_freq, np.abs(sig_fft[:, i]), label=PCA_LABELS[i])
        plt.plot(sample_freq, np.abs(sig_fft_wt[:, i]),
                 label="Approximated " + PCA_LABELS[i])
        if j == 0:
            plt.xlim([0, 3])
        else:
            plt.xlim([0, 1])
            plt.xticks(np.arange(0, 1, 0.1))

        plt.xlabel("Freq [Hz]")
        plt.ylabel("Power")
        plt.legend(loc="best")


# Metrics

### Detection of the peaks

In [ ]:
A10=A10.T
peaks = []
valleys = []
for i in range(len(A10)):
    temp_signal = A10[i][:-1]
    temp_peaks, _ = signal.find_peaks(temp_signal)
    temp_valleys, _ = signal.find_peaks(-temp_signal)
    peaks.append(temp_signal[temp_peaks])
    peaks.append(data_std['Abs Time'].values[temp_peaks])
    valleys.append(temp_signal[temp_valleys])
    valleys.append(data_std['Abs Time'].values[temp_valleys])
    
fig, ax = plt.subplots(len(A10), 1, figsize=(20, 15))
for i in range(1, len(peaks), 2):    
    ax[int(i/2)].plot(data_std['Abs Time'], A10[int(i/2)][:-1])
    ax[int(i/2)].plot(peaks[i], peaks[i-1], 'x')

### Improve peak detection

In [ ]:
avg_peak_dist = []
for i in range(1, len(peaks), 2):    
    avg_peak_dist.append(len(data_std['Abs Time']) / len(peaks[i]))
print(avg_peak_dist)

sign_amp = []
for i in range(1, len(peaks), 2):
    temp_sign_amp = []    
    for j in range(np.min([len(peaks[i]), len(valleys[i])])):
        temp_peaks = np.array(peaks[i-1])
        temp_valleys = np.array(valleys[i-1])
        temp_sign_amp.append(0.5*np.abs(temp_peaks[j] + temp_valleys[j]))
    sign_amp.append(temp_sign_amp)

avg_sign_amp = np.empty(shape=(len(A10)))
for i,x  in enumerate(sign_amp):
    avg_sign_amp[i] = np.mean(x)

height_perc = .6
distance_perc = .7
peaks_refined = []
valleys_refined = []
for i in range(len(A10)):
    temp_signal = A10[i][:-1]
    temp_peaks, _ = signal.find_peaks(temp_signal, height = height_perc * avg_sign_amp[i], distance = distance_perc * avg_peak_dist[i])
    temp_valleys, _ = signal.find_peaks(-temp_signal)
    peaks_refined.append(temp_signal[temp_peaks])
    peaks_refined.append(data_std['Abs Time'].values[temp_peaks])
    valleys_refined.append(temp_signal[temp_valleys])
    valleys_refined.append(data_std['Abs Time'].values[temp_valleys])

fig, ax = plt.subplots(len(A10), 1, figsize=(20, 15))
for i in range(1, len(peaks_refined), 2):    
    ax[int(i/2)].plot(data_std['Abs Time'], A10[int(i/2)][:-1])
    ax[int(i/2)].plot(peaks_refined[i], peaks_refined[i-1], 'x')
    ax[int(i/2)].axhline(avg_sign_amp[int(i/2)] * height_perc, color='g')
    # ax[int(i/2)].plot(valleys_refined[i], valleys_refined[i-1], 'x')

### Generate histogram and perform gaussian fit to estimate RR

In [ ]:
time_dist = []
for i in range(1, len(peaks_refined), 2):
    temp_time_dist = []    
    for j in range(1, len(peaks_refined[i])):
        temp_peaks_time = peaks_refined[i]
        temp_time_dist.append(temp_peaks_time[j]-temp_peaks_time[j-1])
    time_dist.append(temp_time_dist)

time_dist = np.concatenate(time_dist)
print(len(time_dist))
original_mean = np.mean(time_dist)
print(original_mean)
original_std = np.std(time_dist)
print(original_std)

fig, ax = plt.subplots(figsize=(8,6))
bins = ax.hist(x=time_dist)
lower_sigma_tol = 2
upper_sigma_tol = 3
# ax.axvline(np.mean(time_dist) - lower_sigma_tol*original_std, color='r')
# ax.axvline(np.mean(time_dist) + upper_sigma_tol*original_std, color='r')

bin_centers = (bins[1][:-1] + bins[1][1:]) / 2
bin_counts = bins[0]

def myround(x):
    return round(x,1)

ax.set_xticks(bin_centers)
_=ax.set_xticklabels(map(myround,bin_centers))

def my_gaus(x, A, mu, sigma):
    return A * np.exp(-0.5 * ((x-mu)/sigma) ** 2)

params, params_cov = optimize.curve_fit(my_gaus, bin_centers, bin_counts, p0=[1, np.mean(time_dist), np.std(time_dist)], absolute_sigma=True, bounds=(0,[100, 100, 100]))
fit_domain = np.sort(np.random.uniform(np.min(bin_centers), np.max(bin_centers), 1000))
_=ax.plot(fit_domain, my_gaus(fit_domain, *params))

print('Fit parameters:\n',params)
print('Fit parameters errors:\n', np.sqrt(np.diag(params_cov)))

**Alternative method without filter**

When we have no filter we can consider the frequencies' range [0.1,0.4] and compute the respiration rate as the mean of the frequencies of the three max power in the fourier analysis for each of the six principal components

In [ ]:
sample_freqi = (sample_freq > 0.1) & (sample_freq < 0.4)
power_fft = np.abs(sig_fft)

freqi_maxs = np.argmax(power_fft[sample_freqi, :], axis=0)
freq_maxs = sample_freq[sample_freqi][freqi_maxs]
RR = np.mean(freq_maxs)

print(f"RPM is: {RR * 60:.2f}")

# Conclusions

## EXTRA

In [ ]:
#filter signal with Butterworth bandpass filter in [0.1,0.5] Hz range

freq=200
filtered = np.zeros((Nvar,len(data_std)))
sos = signal.butter(4, [0.1, 0.8], 'bandpass', fs=freq, output='sos')
for i in range(Nvar):
    filtered[i] = signal.sosfilt(sos, data_std[labels[i]])
    

In [ ]:
#plot filtered Gyro and Acc
fig, axs = plt.subplots(nrows=Nvar, ncols=2, figsize=(18,Nvar*5))

for i in range(2):
    for j in range(Nvar):
        axs[j][i].plot(data_std["Abs Time"], data_std[labels[j]], label=labels[j])
        axs[j][i].plot(data_std["Abs Time"], filtered[j],         label=("Filtered "+labels[j]))
        if i == 1:
            axs[j][i].set_xlim([20,24])

for ax in axs.flatten():
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Magnitude")
    ax.legend(loc="best")

In [ ]:
#calculate spectrum after filtering
sig_fft_filt = 1j*np.zeros((Nvar, len(data_std)))

for i in range(Nvar):
    sig_fft_filt[i] = fftpack.fft(filtered[i])

power_filt = np.abs(sig_fft_filt)

In [ ]:
#plot filtered spectrum
fig, axs = plt.subplots(nrows=Nvar, ncols=2, figsize=(18,Nvar*5))

for i in range(2):
    for j in range(Nvar):
        axs[j][i].plot(sample_freq, power[j],      label=labels[j])
        axs[j][i].plot(sample_freq, power_filt[j], label=("Filtered "+labels[j]))
        if i == 0:
            axs[j][0].set_xlim([0,3])
        else:
            axs[j][i].set_xlim([0,1])
            axs[j][i].set_xticks(np.arange(0,1,0.1))
        

for ax in axs.flatten():
    ax.set_xlabel("Freq [Hz]")
    ax.set_ylabel("Power")
    ax.legend(loc="best")

In [ ]:
#Filter signal using wavelets
#We used db6 as it is smooth, with a decomposition level=7 
#as we are interested only in low frequencies, and the approximate coefficients at level J
#cover ranges [0, freq/2^(J+1)], so or the last decomposition [0, 0.78125] Hz
lvl = 0
if file_name == "center_sternum.txt":
    lvl = 7
else:
    lvl = 6
    
A7 = np.zeros((Nvar, len(data_std)+1))
for i in range(Nvar):
    coeffs = pywt.wavedec(filtered[i], "db6", level=lvl)
    for l in range(1, lvl+1):
        coeffs[l] = np.zeros_like(coeffs[l])
    A7[i] = pywt.waverec(coeffs, "db6") 
    


In [ ]:
#plot only approximation wavelet
fig, axs = plt.subplots(nrows=Nvar, ncols=2, figsize=(18,Nvar*5))

for i in range(2):
    for j in range(Nvar):
        axs[j][i].plot(data_std["Abs Time"], A7[j][:-1], label=labels[j])
        if i == 1:
            axs[j][i].set_xlim([20,24])
        

for ax in axs.flatten():
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Magnitude")
    ax.legend(loc="best")

In [ ]:
#calculate spectrum after wavelet
sig_fft_wt = 1j*np.zeros((Nvar, len(data_std)))

for i in range(Nvar):
    sig_fft_wt[i] = fftpack.fft(A7[i][:-1])

power_wt = np.abs(sig_fft_wt)

In [ ]:
#plot filtered spectrum
fig, axs = plt.subplots(nrows=Nvar, ncols=2, figsize=(18,Nvar*5))

for i in range(2):
    for j in range(Nvar):
        axs[j][i].plot(sample_freq, power[j],    label=labels[j])
        axs[j][i].plot(sample_freq, power_wt[j], label=("Approximated "+labels[j]))
        if i == 0:
            axs[j][0].set_xlim([0,3])
        else:
            axs[j][i].set_xlim([0,1])
            axs[j][i].set_xticks(np.arange(0,1,0.1))
        

for ax in axs.flatten():
    ax.set_xlabel("Freq [Hz]")
    ax.set_ylabel("Power")
    ax.legend(loc="best")